# Load libraries and start chromedriver

In [1]:
#Start the webdriver
import time
from selenium import webdriver
import re
import pandas as pd
from selenium.common.exceptions import NoSuchElementException
from itertools import *
import numpy
import chardet
import numpy as np

# Load csv

In [2]:
brand_products = pd.read_csv('skincare.csv')
brand_products['brand_product'] = brand_products['brand name'] + ' ' +brand_products['product']
brand_products = brand_products[pd.notnull(brand_products['ingredients'])]

In [3]:
#brand_products.iloc[358]

# Scraping

In [4]:
path_to_chromedriver = '/Users/isabelbeard/Downloads/chromedriver' # change path as needed
browser = webdriver.Chrome(executable_path = path_to_chromedriver)
url = 'http://www.cosdna.com/eng/'
browser.get(url)

In [5]:
%%time
appended_data = []
appended_data_NH = []

for i, product in islice(brand_products.iterrows(), 3552, None):
    time.sleep(7)
    brandName = product['brand name']
    productName = product['product']
    search = browser.find_element_by_name('q')
    search.clear()
    search.send_keys(product['brand_product'])
    search.submit()
    
    try:
        #see if the element exists
        results = browser.find_element_by_class_name("ProdName")
        
        if results.is_displayed():
            
            #check point
            print(str(i), ' ', brandName, ' ', productName, ' exists')
            
            #clicks first search result
            continue_link = results.find_element_by_tag_name('a')
            continue_link.click()
            
            #Find the table in the index page
            table = browser.find_element_by_class_name("iStuffTable")
            
            #If the table exists
            if table.is_displayed():
                
                test_elem = table.find_elements_by_tag_name('tr')
                
                #Find the length of the table
                numberIngredients =  (len(test_elem))

                #Creates a list of items in the width of the table so we can get the right amount each time
                column = browser.find_elements_by_class_name("iStuffList")
                column_len = len(column)
                    
                #We then want to create an index using the number of ingredients in the table
                index = list(range(0,numberIngredients))
                
                #Create an empty dataframe with the index of ingredients and number of columns
                df = pd.DataFrame(index=index, columns=column)
                                
                try:
                    divNo = str(10)
                    test = table.find_element_by_xpath("//*[@id='pagebase']/div["+divNo+"]/div[2]/table/tbody")
                except NoSuchElementException:
                    divNo = str(11)
                    test = table.find_element_by_xpath("//*[@id='pagebase']/div["+divNo+"]/div[2]/table/tbody")
                    
                #iterate through the lengh and width of our dataframe to grab the elements we need in the table
                for x in range(0,len(index)):
                    for y in range(0,len(column)):
                        try:           
                            item = table.find_element_by_xpath("//*[@id='pagebase']/div["+divNo+"]/div[2]/table/tbody/tr["+str(x+1)+"]/td["+str(y+1)+"]")
                            text = item.text
                            
                            #Some basic cleaning
                            textClean = text.replace("\n", ", ")
                            textClean = textClean.replace("‧", "")
                            textClean = textClean.replace(", ~, ", "~")
                            df.iloc[x][y]=textClean
                            
                        except NoSuchElementException:
                            item = table.find_element_by_xpath("//*[@id='pagebase']/div["+divNo+"]/div[2]/table/tbody/tr["+str(x+1)+"]/td[1]")
                            df.iloc[x][y] = item.text
                            
                
                #make sure it's in a  dataframe
                all_ings = pd.DataFrame(df)
                all_ings.columns = all_ings.iloc[0]
                all_ings = all_ings.iloc[1:]
                emptyItemIndexFirst = []
                emptyItemIndex = []
    
                # see if there are any 'no matched results' ingredients
                #We want to get them in the right place so we search for the blanks to replace them with
                for z in range(len(all_ings)):
                    if all_ings.iloc[z][0] == "":
                        emptyItemIndexFirst.append(z)
                    if all_ings.iloc[z][1] == "no matched results":
                        emptyItemIndex.append(z)
                
                #here we sitch the no matched ingredient with it's original empty place
                for a in range(len(emptyItemIndexFirst)):
                    index = emptyItemIndexFirst[a]
                    index2 = emptyItemIndex[a]
                    all_ings.iloc[index][0] = all_ings.iloc[index2][0]
               
            
                #if there are empty items then we trim the dataframe, otherwise it stays the same
                if len(emptyItemIndexFirst) > 0:    
                    all_ings = all_ings[:-len(emptyItemIndexFirst)]
                else:
                    all_ings = all_ings
                    
                ProdTitleName = browser.find_element_by_class_name("ProdTitleName")
                ProdTitleName = ProdTitleName.text
                
                final_df = all_ings       

                final_df.insert(0, 'Order', range(1, 1+len(all_ings)))

                final_df.loc[:,'Product_Name'] = ProdTitleName
                final_df.loc[:,'Brand_Searched'] = brandName
                final_df.loc[:,'Product_Searched'] = productName
                final_df.loc[:,'ID'] = i
    
    except NoSuchElementException:
        productLink = browser.find_element_by_link_text('Analyze Cosmetics')
        productLink.click()
        ingredients = product['ingredients']
        #ingredients = ['marula oil, Aloe Barbadensis Leaf Juice, Maltodextrin, Caprylyl Glycol, Xanthan Gum, Polysorbate 60, Isohexadecane']
        search = browser.find_element_by_name('q')
        search.clear()
        search.send_keys(ingredients)
        search.submit()

        print(str(i), ' ', brandName, ' ', productName, ' searched')


        #Find the table in the index page
        table = browser.find_element_by_class_name("iStuffTable")

        #If the table exists
        if table.is_displayed():
            test_elem = table.find_elements_by_tag_name('tr')

            #Find the length of the table
            numberIngredients =  (len(test_elem))
            if numberIngredients > 0:

                #Creates a list of items in the width of the table so we can get the right amount each time
                column = browser.find_elements_by_class_name("iStuffList")
                column_len = len(column)

                #We then want to create an index using the number of ingredients in the table
                index = list(range(0,numberIngredients))

                #Create an empty dataframe with the index of ingredients and number of columns
                df = pd.DataFrame(index=index, columns=column)

                divNo = str(8)
                test = table.find_element_by_xpath("//*[@id='pagebase']/div["+divNo+"]/div/table/tbody")
                
                
#                print(str(i), ' ', brandName, ' ', productName, ' test 1')
                #iterate through the lengh and width of our dataframe to grab the elements we need in the table
                for x in range(0,len(index)):
                    for y in range(0,len(column)):
                        try:           
                            item = table.find_element_by_xpath("//*[@id='pagebase']/div["+divNo+"]/div/table/tbody/tr["+str(x+1)+"]/td["+str(y+1)+"]")
                            text = item.text
                            #Some basic cleaning
                            textClean = text.replace("\n", ", ")
                            textClean = textClean.replace("‧", "")
                            textClean = textClean.replace(", ~, ", "~")
                            df.iloc[x][y]=textClean

                        except NoSuchElementException:
                            item = table.find_element_by_xpath("//*[@id='pagebase']/div["+divNo+"]/div/table/tbody/tr["+str(x+1)+"]/td[1]")
                            df.iloc[x][y] = item.text        

#                print(str(i), ' ', brandName, ' ', productName, ' test 2')
                #make sure it's in a  dataframe
                all_ings = pd.DataFrame(df)
                all_ings.columns = all_ings.iloc[0]
                all_ings = all_ings.iloc[1:]
                emptyItemIndexFirst = []
                emptyItemIndex = []
#                print(str(i), ' ', brandName, ' ', productName, ' test 3')
                # see if there are any 'no matched results' ingredients
                #We want to get them in the right place so we search for the blanks to replace them with
                
                
                for z in range(len(all_ings)):
                    if all_ings.iloc[z][0] == "":
                        emptyItemIndexFirst.append(z)
                    if all_ings.iloc[z][1] == "no matched results":
                        emptyItemIndex.append(z)
#                print(str(i), ' ', brandName, ' ', productName, ' test 4')
                
                if len(emptyItemIndex) > 0:                
                    #here we sitch the no matched ingredient with it's original empty place
                    for a in range(len(emptyItemIndexFirst)):
                        index = emptyItemIndexFirst[a]
                        index2 = emptyItemIndex[a]
                        all_ings.iloc[index][0] = all_ings.iloc[index2][0]

#                print(str(i), ' ', brandName, ' ', productName, ' test 5')
                #if there are empty items then we trim the dataframe, otherwise it stays the same
                if len(emptyItemIndexFirst) > 0:    
                    all_ings = all_ings[:-len(emptyItemIndexFirst)]
                else:
                    all_ings = all_ings

#                print(str(i), ' ', brandName, ' ', productName, ' test 6')
                final_df = all_ings       

                final_df.insert(0, 'Order', range(1, 1+len(all_ings)))

                final_df.loc[:,'Brand_Searched'] = brandName
                final_df.loc[:,'Product_Name'] = brandName + " " + productName
                final_df.loc[:,'Product_Searched'] = productName
                final_df.loc[:,'ID'] = i
            
            else:
                print(str(i), ' ', brandName, ' ', productName, ' skipped')
                d = {'Order': 'Not here', 'Ingredient': 'Not here', 'Function': 'Not here', 'Acne': 'Not here', 'Irritant': 'Not here', 'Safety': 'Not here', 'ProductName': 'Not here', 'Brand_Searched': brandName, 'Product_Searched': productName}
                index = list(range(0,1))
                final_df = pd.DataFrame(data=d, index = index)
                final_df['ID'] = i
                
    
    appended_data.append(final_df)
    
    productLink = browser.find_element_by_link_text('Product Search')
    productLink.click()

appended_data = pd.concat(appended_data, axis=0)
appended_data = appended_data[['ID', 'Brand_Searched', 'Product_Searched', 'Order', 'Ingredient', 'Function', 'Safety', 'Acne', 'Irritant' ]]

#appended_data_NH.to_csv('180518_02_data_NH.csv', index=False)
#appended_data.to_csv('180518_02_data.csv', index=False)

3613   Avon   Clearskin Blemish Clearing Foaming Cleanser  exists
3614   Avon   Foot Works Intensive Callus Cream  exists
3615   Clarins   Gentle Day Cream, for Sensitive Skin  searched
3616   Chanel   Body Excellence Nourishing and Rejuvenating Hand Cream  exists
3617   Clarins   Super Restorative Day Cream  exists
3618   Clarins   Pure Melt Cleansing Gel, for All Skin Types  searched
3619   Clean & Clear    Advantage 3-in-1 Foaming Acne Wash  exists
3620   Clarins   Extra-Firming Night Cream, Special for Dry Skin  searched
3621   Clarins   Multi-Active Night Youth Recovery Cream for Normal to Combination Skin  exists
3622   Clarins   Super Restorative Night Cream  exists
3623   Clinique   Sparkle Skin Body Exfoliating Cream  exists
3624   Clinique   Super City Block Oil-Free Daily Face Protector Broad Spectrum 40 SPF  exists
3625   Clinique   Self Sun Body Tinted Lotion, Light-Medium  exists
3626   Clinique   After Sun Rescue Balm with Aloe  exists
3627   Clinique   Oil Control Face 

In [7]:
#appended_data = pd.concat(appended_data, axis=0)
#appended_data = appended_data[['ID', 'Brand_Searched', 'Product_Searched', 'Order', 'Ingredient', 'Function', 'Safety', 'Acne', 'Irritant' ]]


#appended_data_NH.to_csv('180519_01_data_NH.csv', index=False)
appended_data.to_csv('180519_05_data.csv', index=False)